In [4]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import progressbar
from time import sleep
from datetime import datetime
import os

movie_dict = {'bitcoin': ['bitcoin since:2022-01-01 until:2022-01-17', 50000]}

In [5]:
today = datetime.today().strftime('%Y%m%d')[2:]+'_'
for index, movie_name in enumerate(movie_dict):
    print(movie_name, '%')
    tweets_list1 = []
    bar = progressbar.ProgressBar(maxval=movie_dict[movie_name][1]+2, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{movie_dict[movie_name][0]}').get_items()): #declare a username
        bar.update(i+1)
        if i>movie_dict[movie_name][1]: #number of tweets you want to scrape
            break
        #print(movie_name, i, tweet)
        tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username]) #declare the attributes to be returned
    tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

    tweets_df1[['Datetime', 'Text']].to_csv(f'{index}.csv')
    bar.finish()

bitcoin %


/Users/aeshwihdi/opt/anaconda3/lib/python3.9/json/decoder.py:353: RuntimeWarning: coroutine 'Twint.main' was never awaited
  obj, end = self.scan_once(s, idx)
[=========================================================================] 100%


In [17]:
import pandas as pd

#when importing empty rows, they are transformed to nan, so we need to drop them here
df = pd.read_csv('0.csv', lineterminator='\n')[['Text']]
df

,Text
0,$KAVA price is rising! • Price (#USD): $ 5.624...
1,"Bitcoin, Crypto And NFTs Are Braced For A Big ..."
2,Chatted up a guy on the chairlift. He's been o...
3,@PeterSchiff I kind of agree that max is a clo...
4,BTC Latest Block Info: Block 719054 holds 292 ...
...,...
49996,@sirsupermario that’s when they all lost 70% o...
49997,@msondergaard2 @CherrySunshyne @Toomanysecurit...
49998,@CryptoCoinHans Ich wünsche nichts 🤣. Glaube n...
49999,"In this article, we compare the hard money of ..."


In [12]:
df.columns

Index(['Unnamed: 0', 'Datetime', 'Text'], dtype='object')

In [18]:
#get rid of links and hashtags
df["text"] = df["Text"].apply(lambda x : ' '.join([s for s in x.split(' ') if s.find('@') == -1 and s.find('www') == -1 and s.find('https') == -1]))

#get rid of non-ascii characters
df = df.replace(r'\W+', ' ', regex=True)
df

,Text,text
0,KAVA price is rising Price USD 5 62400000 Sha...,KAVA price is rising Price USD 5 62400000 Sha...
1,Bitcoin Crypto And NFTs Are Braced For A Big W...,Bitcoin Crypto And NFTs Are Braced For A Big W...
2,Chatted up a guy on the chairlift He s been on...,Chatted up a guy on the chairlift He s been on...
3,PeterSchiff I kind of agree that max is a clo...,I kind of agree that max is a clown who comes ...
4,BTC Latest Block Info Block 719054 holds 292 t...,BTC Latest Block Info Block 719054 holds 292 t...
...,...,...
49996,sirsupermario that s when they all lost 70 of...,that s when they all lost 70 of their bitcoin ...
49997,msondergaard2 CherrySunshyne Toomanysecurity ...,This is so delusional Countries are mining bit...
49998,CryptoCoinHans Ich wünsche nichts Glaube nur ...,Ich wünsche nichts Glaube nur dass sich bitcoi...
49999,In this article we compare the hard money of t...,In this article we compare the hard money of t...


In [21]:
# python3 -m spacy download en_core_web_sm

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("spacytextblob")

df['sentiment'] = df['text'].apply(lambda x : nlp(x)._.polarity)
df_sentiment = df.sort_values('sentiment').reset_index(drop=True)
df_sentiment

,Text,text,sentiment
0,jordanbpeterson Talk about bitcoin and Trudea...,Talk about bitcoin and Trudeau This is boring,-1.0
1,bitcoin is so boring just dump this shitcoin ...,bitcoin is so boring just dump this shitcoin ...,-1.0
2,Bitcoin why so boring,Bitcoin why so boring,-1.0
3,Worst part of bitcoin dips is seeing the fiat ...,Worst part of bitcoin dips is seeing the fiat ...,-1.0
4,Bitcoin BTC Is Off to Worst Annual Start Since...,Bitcoin BTC Is Off to Worst Annual Start Since...,-1.0
...,...,...,...
49996,Wonderful Bitcoin hashimubn FlorianThauvin The...,Wonderful Bitcoin,1.0
49997,mikealfred There is Bitcoin And there are shi...,There is Bitcoin And there are shitcoins compe...,1.0
49998,Join Paxful best platform for crypto crypto bi...,Join Paxful best platform for crypto crypto bi...,1.0
49999,LtsGOBrandon found bitcoin in a User vault at ...,LtsGOBrandon found bitcoin in a User vault at ...,1.0


In [22]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

#adding a counter to check the progress of the algo while it runs
global counter
counter = 0
def preprocess(sentence, stemming=False, lemmatizing=False):
  global counter
  counter += 1
  if counter % 100 == 0:
    pass
    #print(counter)

  #clean as much as possible, but not apply strong editing to the text, yet
  sentence=str(sentence)
  tokenizer = RegexpTokenizer(r'\w+')

  sentence = sentence.lower()
  sentence=sentence.replace('{html}',"") 
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', sentence)
  rem_url=re.sub(r'http\S+', '',cleantext)
  rem_num = re.sub('[0-9]+', '', rem_url)
  tokens = tokenizer.tokenize(rem_num)
  
  filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  
  if stemming == True and lemmatizing == False:
    stem_words=[stemmer.stem(w) for w in filtered_words]
    return " ".join(stem_words)

  if stemming == False and lemmatizing == True:
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    return " ".join(lemma_words)

  if stemming == True and lemmatizing == True:
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)
  
  #at the end of the algo we return filtered words
  return " ".join(filtered_words)

#preprocess the sentiment text
df_sentiment['text'] = df_sentiment['text'].apply(lambda x: preprocess(x, stemming=False, lemmatizing=True))
df_sentiment

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aeshwihdi/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aeshwihdi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Text,text,sentiment
0,jordanbpeterson Talk about bitcoin and Trudea...,talk bitcoin trudeau boring,-1.0
1,bitcoin is so boring just dump this shitcoin ...,bitcoin boring dump shitcoin already,-1.0
2,Bitcoin why so boring,bitcoin boring,-1.0
3,Worst part of bitcoin dips is seeing the fiat ...,worst part bitcoin dip seeing fiat gargoyle em...,-1.0
4,Bitcoin BTC Is Off to Worst Annual Start Since...,bitcoin btc worst annual start since dawn cryp...,-1.0
...,...,...,...
49996,Wonderful Bitcoin hashimubn FlorianThauvin The...,wonderful bitcoin,1.0
49997,mikealfred There is Bitcoin And there are shi...,bitcoin shitcoins competing best shitcoin,1.0
49998,Join Paxful best platform for crypto crypto bi...,join paxful best platform crypto crypto bitcoi...,1.0
49999,LtsGOBrandon found bitcoin in a User vault at ...,ltsgobrandon found bitcoin user vault location...,1.0


In [38]:
df_neg = df_sentiment[df_sentiment['sentiment'] < 0]
df_pos = df_sentiment[df_sentiment['sentiment'] > 0]
print('The number of neg tweets is ', len(df_neg))
print('The number of pos tweets is ', len(df_pos))
print('            ---                ')
if len(df_pos) > len(df_neg):
    print('Positive Teweets')
else:
    print('Negative Teweets')

The number of neg tweets is  5327
The number of pos tweets is  16382
            ---                
Positive Teweets


In [28]:
positive_words = pd.DataFrame([dict(Counter(' '.join(df_pos['text'].values.tolist()).split(' ')))]).T.sort_values(0, ascending=False)[0:100].index

negative_words = pd.DataFrame([dict(Counter(' '.join(df_neg['text'].values.tolist()).split(' ')))]).T.sort_values(0, ascending=False)[0:100].index

NameError: name 'Counter' is not defined